# Genetic Algorithm Study with a tutorial

## Initial population set

In [2]:
import math
import random

def init_generation(chromo_len, pop_size, generation,
                    next_generation, fitness, selector_prob):
    '''Initial first generation by random'''
    n = 2 ** chromo_len -1  # max value of chromosome
    for i in range(pop_size):
        generation.append((random.randint(0, n), random.randint(0, n)))
        next_generation.append((0, 0))
        fitness.append(0)
        selector_prob.append(0)   

## Population definition

In [13]:
age = 0                 # the count of generation
pop_size = 50         # totoal population of a generation
gen_max = 159            # max amount of generation
chromo_len = 25        # length of chromosome
cross_prob = 0.8        # probability of crossover
mutat_prob = 0.1        # probability of mutation

generation = []         # current generation of individuals
next_generation = []    # next generation of individuals
fitness = []            # fitness value of each individual in current generation
selector_prob = []      # probability of each individual to be selected

# the best individual of that generation
elitist = {'chromosome':(0, 0), 'fitness':0, 'age':0}

# Initial generation params
init_generation(chromo_len, pop_size, generation,
                next_generation, fitness, selector_prob)

## Define encoding and decoding function

In [4]:
def decode(interval, chromosome, chromo_len):
    '''Project chromosome to feasible interval'''
    delta = interval[1] - interval[0]
    ratio = float(2 ** chromo_len -1)
    return (interval[0] + chromosome * delta / ratio)    

## Define fitness function

In [10]:
def fitness_func(chromo_x, chromo_y):
    '''Calculate the fitness value of a chromosome pair'''
    interval = [-10.0, 10.0]
    x = decode(interval, chromo_x, chromo_len)
    y = decode(interval, chromo_y, chromo_len)
    nom = lambda x,y: math.sin(math.sqrt(x*x + y*y)) ** 2 - 0.5
    denom = lambda x,y: (1 + 0.001 * (x*x + y*y)) ** 2
    f = lambda x,y: 0.5 - nom(x,y) / denom(x,y)
    return f(x,y)

def evaluate(pop_size, fitness, generation, selector_prob):
    '''Evalutate the selection probability of each individual'''
    for i in range(pop_size):
        fitness[i] = fitness_func(generation[i][0], generation[i][1])
    
    total = sum(fitness)
    for i in range(pop_size):
        selector_prob[i] = fitness[i] / total
    
    # For Roulette Selection
    for i in range(1, pop_size):
        selector_prob[i] += selector_prob[i-1] 
        

## Define genetic operation

In [11]:
def select(selector_prob):
    t = random.random()
    i = 0 
    for p in selector_prob:
        if p > t:
            break
        i += 1
    return i

def cross (chromo_len, cross_prob, chrom1, chrom2):
    p = random.random ()
    n = 2 ** chromo_len -1
    if chrom1 != chrom2 and p < cross_prob:
        t = random.randint (1, chromo_len - 1)
        #print bin(chrom1),bin(chrom2)
        mask = n << t
        (r1, r2) = (chrom1 & mask, chrom2 & mask)
        #print mask,bin(mask)
        #print bin(r1),bin(r2)
        mask = n >> (chromo_len - t)
        (l1, l2) = (chrom1 & mask, chrom2 & mask)
        #print mask,bin(mask)
        #print bin(l1), bin(l2)
        
        (chrom1, chrom2) = (r1 + l2, r2 + l1)
    return (chrom1, chrom2)

    
def mutate(chromo, chromo_len):
    p = random.random()
    mutat_prob = 1
    if p < mutat_prob:
        t = random.randint(1, chromo_len)
        mask1 = 1 << (t - 1)
        mask2 = chromo & mask1
        if mask2 > 0:
            chromo = chromo & (~mask2)
        else:
            chromo = chromo ^ mask1
    return chromo

## Evolution process

In [14]:
for k in range(gen_max):
    # Evaluate selection prob of current generation
    evaluate(pop_size, fitness, generation, selector_prob)
    
    # Evolve with genetic operations
    i = 0
    while True:
        # Select individual
        id1 = select(selector_prob)
        id2 = select(selector_prob)
        (id1_x, id1_y) = (generation[id1][0], generation[id1][1])
        (id2_x, id2_y) = (generation[id2][0], generation[id2][1])
        
        # Perform cross
        (id1_x, id2_x) = cross(chromo_len, cross_prob, id1_x, id2_x)
        (id1_y, id2_y) = cross(chromo_len, cross_prob, id1_y, id2_y)
        
        # Perform mutate
        id1_val = (mutate(id1_x, chromo_len), mutate(id1_y, chromo_len))
        id2_val = (mutate(id2_x, chromo_len), mutate(id2_y, chromo_len))
         
        # Generate next generation    
        next_generation[i] = id1_val
        next_generation[i + 1] = id2_val
        i += 2
        
        # Check termination condition
        if i >= pop_size:
            break
    
    # Replace the current generation with next one
    generation = list(next_generation)
    print (k, max(fitness), sum(fitness)/pop_size, min(fitness)) 

(0, 0.9888036105879638, 0.5959338001626281, 0.05703049890797024)
(1, 0.9884689801632246, 0.7077176869169227, 0.039464197647180566)
(2, 0.9873492899438738, 0.6988722061506727, 0.11733529050064961)
(3, 0.984697096462708, 0.7400217755821714, 0.08868084626548683)
(4, 0.9901377381782459, 0.5367937755341965, 0.03513309249810731)
(5, 0.9900418945692753, 0.6197784394478498, 0.0025341751262340506)
(6, 0.9901124032399184, 0.6790398162241358, 0.022820484236012295)
(7, 0.9879907611945147, 0.6376520519641362, 0.05966574825460208)
(8, 0.9887707763392004, 0.7215927305043857, 0.03034337845269952)
(9, 0.9895901640865001, 0.7667585747179804, 0.26069187690646956)
(10, 0.9898128270461737, 0.719887414820263, 0.032172616952928834)
(11, 0.9902082254752214, 0.7056220035320965, 0.021600318830279797)
(12, 0.9902301272174032, 0.7231300291404702, 0.023243142313606358)
(13, 0.9888525497275408, 0.752899359270276, 0.08217912376102804)
(14, 0.9902101577282816, 0.7037504145153474, 0.019754505776528652)
(15, 0.98899201

In [75]:
import numpy as np
inx = np.argmax(np.array(fitness))
print fitness[inx],max(fitness)

0.93736227582 0.93736227582
